In [2]:
from data_prep import SystemIdentDataset, ControllerDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import copy
import math

train_dataset = SystemIdentDataset(train=True)
val_dataset = SystemIdentDataset(train=False)
train_dloader = DataLoader(train_dataset, batch_size=256,shuffle=True)
val_dloader = DataLoader(val_dataset, batch_size=256, shuffle=True)

def evaluate(model, loss_function, val_loader):
    c_error = 0.0
    cos_sim = torch.nn.CosineSimilarity(dim=1)
    running_loss = 0.0
    batch_count = 0
    for _, example in enumerate(tqdm(val_loader), 0):
        inputs,label = example

        with torch.no_grad():
            outputs = model(inputs)
        
        loss = loss_function(outputs, label)
        
        c_error += torch.sum(cos_sim(outputs,label)).item()/val_dloader.batch_size
        running_loss += loss.item()
        batch_count += 1
    
    c_error /= batch_count
    
    return running_loss, c_error

def train(model, num_epochs, loss_function, optimizer, train_loader, val_loader):
    best_loss = []
    best_cerror = []
    val_loss, c_error = evaluate(model, loss_function, val_loader)
    print(f"Initial validation loss: {val_loss}, cosine error: {c_error}")
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}:")
        for _, example in enumerate(tqdm(train_loader), 0):
            inputs,label = example
            
            outputs = model(inputs)

            loss = loss_function(outputs, label)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()
        val_loss, c_error = evaluate(model, loss_function, val_loader)
        print(f"validation loss: {val_loss}, cosine error: {c_error}")

        if epoch < 5:
            best_loss.append((copy.deepcopy(model), val_loss, c_error))
            best_cerror.append((copy.deepcopy(model), val_loss,c_error))
        else:
            for i, entry in enumerate(best_loss):
                m,l,c = entry
                if val_loss < l:
                    best_loss[i] = (copy.deepcopy(model), val_loss, c_error)
                    break
            for i, entry in enumerate(best_cerror):
                m,l,c = entry
                if c_error > c:
                    best_cerror[i] = (copy.deepcopy(model), val_loss, c_error)
                    break
    return best_loss + best_cerror

In [3]:
%%capture
import numpy as np
# Define the model
model = torch.nn.Sequential(
    torch.nn.Linear(5, 32, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 32, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 4, dtype=torch.float64),
)

# Define the loss function
loss_fn = torch.nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())
best_models = train(model=model, num_epochs=50, loss_function=loss_fn, optimizer=optimizer, train_loader=train_dloader, val_loader=val_dloader)
for i, model in enumerate(best_models):
    weights, loss, c_error = model
    print(f"Model {i}: loss: {loss}, cosine error: {c_error}")


In [4]:
for i, model in enumerate(best_models):
    weights, loss, c_error = model
    print(f"Model {i}: loss: {loss}, cosine error: {c_error}")

Model 0: loss: 1.8170448759788496, cosine error: 0.9950331449142636
Model 1: loss: 1.92613692305989, cosine error: 0.9933338245077817
Model 2: loss: 1.9410125440446435, cosine error: 0.9937841096484955
Model 3: loss: 2.751595918908532, cosine error: 0.9939130181530594
Model 4: loss: 2.853689677260321, cosine error: 0.9935595464869591
Model 5: loss: 1.8170448759788496, cosine error: 0.9950331449142636
Model 6: loss: 2.751595918908532, cosine error: 0.9939130181530594
Model 7: loss: 1.9410125440446435, cosine error: 0.9937841096484955
Model 8: loss: 1.92613692305989, cosine error: 0.9933338245077817
Model 9: loss: 3.619384600129787, cosine error: 0.9907320776721391


In [12]:
# Now learn to drive emulated plant from state Zo to Zd in K steps where K is a hyperparameter
import math
from data_prep import SystemIdentDataset, ControllerDataset

train_dataset = ControllerDataset(num_examples=1000000)
val_dataset = ControllerDataset(num_examples=20000)
train_dloader = DataLoader(train_dataset, batch_size=256,shuffle=True)
val_dloader = DataLoader(val_dataset, batch_size=256, shuffle=True)

class ControllerTrainedEnclosure(torch.nn.Module):
    def __init__(self, emulator_network, K):
        super(self.__class__, self).__init__()
        emulator_network.requires_grad=False 
        self.system_emulator = emulator_network
        self.K = K
        self.network = torch.nn.Sequential(
                torch.nn.Linear(4, 32, dtype=torch.float64),
                torch.nn.Tanh(),
                torch.nn.Linear(32, 64, dtype=torch.float64),
                torch.nn.Tanh(),
                torch.nn.Linear(64, 64, dtype=torch.float64),
                torch.nn.Tanh(),
                torch.nn.Linear(64, 32, dtype=torch.float64),
                torch.nn.Tanh(),
                torch.nn.Linear(32, 1, dtype=torch.float64),
                )

    def forward(self, x):
        for _ in range(self.K):
            xnew = torch.zeros_like(x)
            u = self.network(x)
            xnew[:,0] = u.squeeze()
            xnew[:,1:] = x[:,1:]
            
            dx = self.system_emulator(xnew)
            x = x + dx
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=math.inf)
        return x



In [13]:
import pickle
with open('./best_emulator_model.pkl', 'rb') as f:
    emulator = pickle.load(f)
model = ControllerTrainedEnclosure(emulator, K=30)

# Define the loss function
loss_fn = torch.nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())
best_models = train(model=model, num_epochs=5, loss_function=loss_fn, optimizer=optimizer, train_loader=train_dloader, val_loader=val_dloader)
for i, model in enumerate(best_models):
    weights, loss, c_error = model
    print(f"Model {i}: loss: {loss}, cosine error: {c_error}")

100%|██████████| 79/79 [00:02<00:00, 30.28it/s]


Initial validation loss: 11334.112304635986, cosine error: 0.12409789919147671
Epoch 1:


100%|██████████| 79/79 [00:02<00:00, 30.11it/s]


validation loss: 0.001232625359249112, cosine error: 0.9889218624629882
Epoch 2:


100%|██████████| 79/79 [00:02<00:00, 30.70it/s]


...

validation loss: 0.00016930437322523113, cosine error: 0.9889237376109816
Epoch 4:


100%|██████████| 79/79 [00:02<00:00, 28.74it/s]


validation loss: 2.8472482622639886e-05, cosine error: 0.9889239876534487
Epoch 5:


100%|██████████| 79/79 [00:02<00:00, 30.23it/s]

validation loss: 0.05117354786361502, cosine error: 0.9888141661612175
Model 0: loss: 0.001232625359249112, cosine error: 0.9889218624629882
Model 1: loss: 0.0012206012526667222, cosine error: 0.988921066165921
Model 2: loss: 0.00016930437322523113, cosine error: 0.9889237376109816
Model 3: loss: 2.8472482622639886e-05, cosine error: 0.9889239876534487
Model 4: loss: 0.05117354786361502, cosine error: 0.9888141661612175
Model 5: loss: 0.001232625359249112, cosine error: 0.9889218624629882
Model 6: loss: 0.0012206012526667222, cosine error: 0.988921066165921
Model 7: loss: 0.00016930437322523113, cosine error: 0.9889237376109816
Model 8: loss: 2.8472482622639886e-05, cosine error: 0.9889239876534487
Model 9: loss: 0.05117354786361502, cosine error: 0.9888141661612175


In [17]:
with open('./controller_model.pkl', 'wb') as f:
    pickle.dump(best_models[8][0].network, f)
